In [1]:
import pandas as pd 
import os 

import numpy as np
import chart_studio.plotly as py 
import seaborn as sns
import plotly.express as px
import cufflinks as cf
%matplotlib inline

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected = True)
cf.go_offline()

## Merging 12 months files in to single file

In [2]:
files = [file for file in os.listdir('./Sales_Data/')]
files

all_months_data = pd.DataFrame()

for file in files:
    df = pd.read_csv("./Sales_Data/"+file)
    all_months_data = pd.concat([all_months_data, df])
    
all_months_data.to_csv("all_data.csv", index=False)

In [3]:
all_data=pd.read_csv("all_data.csv")
all_data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,NaN,NaN,NaN,NaN,NaN,NaN
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"


## Clean up the data

##### Find a NaN value in DF  

In [4]:
nan_df = all_data[all_data.isna().any(axis=1)]
nan_df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
1,NaN,NaN,NaN,NaN,NaN,NaN
356,NaN,NaN,NaN,NaN,NaN,NaN
735,NaN,NaN,NaN,NaN,NaN,NaN
1433,NaN,NaN,NaN,NaN,NaN,NaN
1553,NaN,NaN,NaN,NaN,NaN,NaN


##### Droping NaN value  

In [5]:
all_data = all_data.dropna(how='all')

##### Finding OR datas 

In [6]:
temp_df = all_data[all_data['Order Date'].str[0:2] == 'Or']
temp_df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
519,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
1149,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
1155,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
2878,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
2893,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address


##### Deleting OR datas 

In [7]:
df = all_data[all_data['Order Date'].str[0:2] != 'Or']

### Add month column

In [8]:
import warnings
warnings.filterwarnings('ignore')

df['Month']= df['Order Date'].str[0:2]
df['Month']= df['Month'].astype('int32') 
df

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001",4
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215",4
3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001",4
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001",4
5,176561,Wired Headphones,1,11.99,04/30/19 09:27,"333 8th St, Los Angeles, CA 90001",4
...,...,...,...,...,...,...,...
186845,259353,AAA Batteries (4-pack),3,2.99,09/17/19 20:56,"840 Highland St, Los Angeles, CA 90001",9
186846,259354,iPhone,1,700,09/01/19 16:00,"216 Dogwood St, San Francisco, CA 94016",9
186847,259355,iPhone,1,700,09/23/19 07:39,"220 12th St, San Francisco, CA 94016",9
186848,259356,34in Ultrawide Monitor,1,379.99,09/19/19 17:30,"511 Forest St, San Francisco, CA 94016",9


##### Convert columns to the correct type of data

In [9]:
df['Quantity Ordered'] = pd.to_numeric(df['Quantity Ordered'])
df['Price Each'] = pd.to_numeric(df['Price Each'] )

##### Add sales column 

In [10]:
df['Sales'] = df["Quantity Ordered"] * df['Price Each']

#### Price level column 

In [11]:
def price_level(x,h,l):
    if x > h:
        return 'High'
    if x < l:
        return 'Low'
    return 'Medium'

# Set high and low price level
hi = 500
low = 50

df['Price Level'] = df['Price Each'].apply(price_level, args=[hi, low])
df

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,Sales,Price Level
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001",4,23.90,Low
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215",4,99.99,Medium
3,176560,Google Phone,1,600.00,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001",4,600.00,High
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001",4,11.99,Low
5,176561,Wired Headphones,1,11.99,04/30/19 09:27,"333 8th St, Los Angeles, CA 90001",4,11.99,Low
...,...,...,...,...,...,...,...,...,...
186845,259353,AAA Batteries (4-pack),3,2.99,09/17/19 20:56,"840 Highland St, Los Angeles, CA 90001",9,8.97,Low
186846,259354,iPhone,1,700.00,09/01/19 16:00,"216 Dogwood St, San Francisco, CA 94016",9,700.00,High
186847,259355,iPhone,1,700.00,09/23/19 07:39,"220 12th St, San Francisco, CA 94016",9,700.00,High
186848,259356,34in Ultrawide Monitor,1,379.99,09/19/19 17:30,"511 Forest St, San Francisco, CA 94016",9,379.99,Medium


## What was the best month ?

In [12]:
results = df.groupby('Month').sum()
results = results.drop(columns='Price Each')
results = results.round(decimals=2)
results

,Quantity Ordered,Sales
Month,,
1,10903,1822256.73
2,13449,2202022.42
3,17005,2807100.38
4,20558,3390670.24
5,18667,3152606.75
6,15253,2577802.26
7,16072,2647775.76
8,13448,2244467.88
9,13109,2097560.13


#### Creating months array

In [13]:
import datetime

m=np.arange(1,13,1).astype('str')
months=[]

for i in m:
    data = datetime.datetime.strptime(i, '%m')
    months.append(data.strftime('%B'))

In [14]:
fig = px.bar(df, x=months, y=results['Sales'],
             color=results['Sales'],
             color_continuous_scale='sunsetdark')

fig.update_traces(texttemplate = '%{value:.2s}', textposition = 'outside')

fig.update_layout(title='Sales in every month',
                  xaxis_title='Months',
                  yaxis_title='Sales [$]',
                  xaxis_tickangle = -45)

#Saving figure 
fig.write_html('Monthly_Sale.html')

fig.show()

### Which product sells the most

In [15]:
results = df.groupby('Product').sum()
results.sort_values(by=['Quantity Ordered'],inplace=True, ascending=False)

products = results.index.tolist()
results

,Quantity Ordered,Price Each,Month,Sales
Product,,,,
AAA Batteries (4-pack),31017,61716.59,146370,92740.83
AA Batteries (4-pack),27635,79015.68,145558,106118.40
USB-C Charging Cable,23975,261740.85,154819,286501.25
Lightning Charging Cable,23217,323787.10,153092,347094.15
Wired Headphones,20557,226395.18,133397,246478.43
Apple Airpods Headphones,15661,2332350.00,109477,2349150.00
Bose SoundSport Headphones,13457,1332366.75,94113,1345565.43
27in FHD Monitor,7550,1125974.93,52558,1132424.50
iPhone,6849,4789400.00,47941,4794300.00


In [16]:
fig = px.bar(df, x=products, y=results['Quantity Ordered'],
             color=results['Quantity Ordered'],
             color_continuous_scale='sunsetdark')

fig.update_traces(texttemplate = '%{value:.2s}', textposition = 'outside')

fig.update_layout(title='Best selling products',
                  xaxis_title='Products',
                  yaxis_title='Quantity',
                  xaxis_tickangle = -45)

### Add City column 

In [17]:
def get_city(x):
    return ( x.split(',')[1] + " " + x.split(',')[2].split(' ')[1]) 
def get_state(x):
    return x.split(',')[2].split(' ')[1]

df['City'] = df['Purchase Address'].apply(get_city)
df['State'] = df['Purchase Address'].apply(get_state)
df

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,Sales,Price Level,City,State
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001",4,23.90,Low,Dallas TX,TX
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215",4,99.99,Medium,Boston MA,MA
3,176560,Google Phone,1,600.00,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001",4,600.00,High,Los Angeles CA,CA
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001",4,11.99,Low,Los Angeles CA,CA
5,176561,Wired Headphones,1,11.99,04/30/19 09:27,"333 8th St, Los Angeles, CA 90001",4,11.99,Low,Los Angeles CA,CA
...,...,...,...,...,...,...,...,...,...,...,...
186845,259353,AAA Batteries (4-pack),3,2.99,09/17/19 20:56,"840 Highland St, Los Angeles, CA 90001",9,8.97,Low,Los Angeles CA,CA
186846,259354,iPhone,1,700.00,09/01/19 16:00,"216 Dogwood St, San Francisco, CA 94016",9,700.00,High,San Francisco CA,CA
186847,259355,iPhone,1,700.00,09/23/19 07:39,"220 12th St, San Francisco, CA 94016",9,700.00,High,San Francisco CA,CA
186848,259356,34in Ultrawide Monitor,1,379.99,09/19/19 17:30,"511 Forest St, San Francisco, CA 94016",9,379.99,Medium,San Francisco CA,CA


### Witch city had the highest number of sales?  

In [18]:
sale = df.groupby('City').sum()
sale = sale.sort_values(by='Sales', ascending = False)
sale

,Quantity Ordered,Price Each,Month,Sales
City,,,,
San Francisco CA,50239,8.211462e+06,315520,8.262204e+06
Los Angeles CA,33289,5.421435e+06,208325,5.452571e+06
New York City NY,27932,4.635371e+06,175741,4.664317e+06
Boston MA,22528,3.637410e+06,141112,3.661642e+06
Atlanta GA,16602,2.779908e+06,104794,2.795499e+06
Dallas TX,16730,2.752628e+06,104620,2.767975e+06
Seattle WA,16553,2.733296e+06,104941,2.747755e+06
Portland OR,11303,1.860558e+06,70621,1.870732e+06
Austin TX,11153,1.809874e+06,69829,1.819582e+06


In [19]:
cities = sale.index.tolist()


fig = px.bar(x=cities, y=sale['Sales'],
             color=sale['Sales'],
             color_continuous_scale='sunsetdark')
fig.update_traces(texttemplate = '%{value:.2s}', textposition = 'outside')
fig.update_layout(title='Best selling states',
                  xaxis_title='States',
                  yaxis_title='Sales [$]',
                  xaxis_tickangle = -45)

## Which state sells the most ?

#### Downloading each state coordinates using Beautiful Soup  

In [20]:
import requests
from bs4 import BeautifulSoup as bs
import re

# Load web page content
r = requests.get('https://developers.google.com/public-data/docs/canonical/states_csv')

print('Status code:',r.status_code)
# Convert to Beautiful Soup object
soup = bs(r.content)

Status code: 200


In [21]:
table = soup.find('table')

In [22]:
tr = table.select('tr')

#### Getting column names 

In [23]:
columns = soup.select('th', { 'scope' : 'col' })
col=[]
for column in columns:
    col.append(column.string)
col

['state', 'latitude', 'longitude', 'name']

#### Getting rows 

In [24]:
lat =[]
lon =[]
state =[] 
for i in range(1,len(tr)):
    state.append(tr[i].select('td')[0].string)
    lat.append(tr[i].select('td')[1].string)        
    lon.append(tr[i].select('td')[2].string)        

In [25]:
coordinates = {'State':state,
               'lat':lat,
               'lon':lon}
df_coordinates = pd.DataFrame(coordinates)
df_coordinates.to_excel('USA_States_Coordinates.xlsx', index=False)

### Merging Coordinates to States DataFrame 

In [26]:
df_sale_by_state = df.groupby('State').sum()


df_sale_by_state = df_sale_by_state.reset_index()

print("States DF shape: ", df_sale_by_state.shape)
print('Klumny:', df_sale_by_state.columns)
df_sale_by_state

States DF shape:  (8, 5)
Klumny: Index(['State', 'Quantity Ordered', 'Price Each', 'Month', 'Sales'], dtype='object')


,State,Quantity Ordered,Price Each,Month,Sales
0,CA,83528,1.363290e+07,523845,1.371477e+07
1,GA,16602,2.779908e+06,104794,2.795499e+06
2,MA,22528,3.637410e+06,141112,3.661642e+06
3,ME,2750,4.471893e+05,17144,4.497583e+05
4,NY,27932,4.635371e+06,175741,4.664317e+06
5,OR,11303,1.860558e+06,70621,1.870732e+06
6,TX,27883,4.562501e+06,174449,4.587557e+06
7,WA,16553,2.733296e+06,104941,2.747755e+06


In [27]:
print("Coordinates DF shape: ", df_coordinates.shape)
print('Klumny:', df_coordinates.columns)
df_coordinates.head()

Coordinates DF shape:  (52, 3)
Klumny: Index(['State', 'lat', 'lon'], dtype='object')


,State,lat,lon
0,AK,63.588753,-154.493062
1,AL,32.318231,-86.902298
2,AR,35.20105,-91.831833
3,AZ,34.048928,-111.093731
4,CA,36.778261,-119.417932


In [28]:
df_merged = pd.merge(df_sale_by_state, df_coordinates)

# Save States DataFrame to CSV
df_merged.to_csv('Sales_bystate.csv', index=False)

# Which state sells the most ?

In [29]:
df_map = pd.read_csv('Sales_bystate.csv')
df_map

,State,Quantity Ordered,Price Each,Month,Sales,lat,lon
0,CA,83528,1.363290e+07,523845,1.371477e+07,36.778261,-119.417932
1,GA,16602,2.779908e+06,104794,2.795499e+06,32.157435,-82.907123
2,MA,22528,3.637410e+06,141112,3.661642e+06,42.407211,-71.382437
3,ME,2750,4.471893e+05,17144,4.497583e+05,45.253783,-69.445469
4,NY,27932,4.635371e+06,175741,4.664317e+06,43.299428,-74.217933
5,OR,11303,1.860558e+06,70621,1.870732e+06,43.804133,-120.554201
6,TX,27883,4.562501e+06,174449,4.587557e+06,31.968599,-99.901813
7,WA,16553,2.733296e+06,104941,2.747755e+06,47.751074,-120.740139


In [30]:
import plotly.express as px
import plotly.graph_objects as go 


fig = go.Figure(data=go.Choropleth(
    locations=df_map['State'], # Spatial coordinates
    z = df_map['Sales'].astype(float), 
    locationmode = 'USA-states', 
    colorscale = 'sunsetdark',
    colorbar_title = "Millions USD",
))

fig.update_layout(
    title={
        'text': "Best selling states",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    geo_scope='usa', 
)

fig.show()

### At what time do we sell the most ? 

In [31]:
df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Hour'] = df['Order Date'].dt.hour
df['Minute'] = df['Order Date'].dt.minute
df

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,Sales,Price Level,City,State,Hour,Minute
0,176558,USB-C Charging Cable,2,11.95,2019-04-19 08:46:00,"917 1st St, Dallas, TX 75001",4,23.90,Low,Dallas TX,TX,8,46
2,176559,Bose SoundSport Headphones,1,99.99,2019-04-07 22:30:00,"682 Chestnut St, Boston, MA 02215",4,99.99,Medium,Boston MA,MA,22,30
3,176560,Google Phone,1,600.00,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",4,600.00,High,Los Angeles CA,CA,14,38
4,176560,Wired Headphones,1,11.99,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",4,11.99,Low,Los Angeles CA,CA,14,38
5,176561,Wired Headphones,1,11.99,2019-04-30 09:27:00,"333 8th St, Los Angeles, CA 90001",4,11.99,Low,Los Angeles CA,CA,9,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...
186845,259353,AAA Batteries (4-pack),3,2.99,2019-09-17 20:56:00,"840 Highland St, Los Angeles, CA 90001",9,8.97,Low,Los Angeles CA,CA,20,56
186846,259354,iPhone,1,700.00,2019-09-01 16:00:00,"216 Dogwood St, San Francisco, CA 94016",9,700.00,High,San Francisco CA,CA,16,0
186847,259355,iPhone,1,700.00,2019-09-23 07:39:00,"220 12th St, San Francisco, CA 94016",9,700.00,High,San Francisco CA,CA,7,39
186848,259356,34in Ultrawide Monitor,1,379.99,2019-09-19 17:30:00,"511 Forest St, San Francisco, CA 94016",9,379.99,Medium,San Francisco CA,CA,17,30


In [32]:
df_time = df.groupby("Hour").sum()
hours = df_time.index.to_list()

fig = px.bar(df_time, x=hours, y='Sales',
             title = 'Sales in each hour',
            color='Sales',
            color_continuous_scale='sunsetdark')

fig.update_layout(xaxis = dict(
        tickmode = 'linear',
        dtick = 1,
        title='Hours'),
                 yaxis = dict(
        title = "Sales [$]"))



fig.show()             

## What product are most often sold together ?
#### Im creating a DataFrame containing only elements with duplicated Order ID.

In [33]:
df_duplicated = df[df['Order ID'].duplicated(keep=False)]
df_duplicated

df_duplicated['Duplicated'] = df_duplicated.groupby('Order ID')['Product'].transform(lambda x: ',' .join(x))

df_duplicated = df_duplicated[['Order ID','Duplicated']].drop_duplicates()

#### Counting unique pairs  

In [34]:
from itertools import combinations
from collections import Counter


n = 2 # nuber of combination // 2 - pairs , 3 - trio

count = Counter()
for row in df_duplicated['Duplicated']:
    row_list = row.split(',')
    count.update(Counter(combinations(row_list,n)))

    
for key, value in count.most_common(10):
    print(key, value)


('iPhone', 'Lightning Charging Cable') 1005
('Google Phone', 'USB-C Charging Cable') 987
('iPhone', 'Wired Headphones') 447
('Google Phone', 'Wired Headphones') 414
('Vareebadd Phone', 'USB-C Charging Cable') 361
('iPhone', 'Apple Airpods Headphones') 360
('Google Phone', 'Bose SoundSport Headphones') 220
('USB-C Charging Cable', 'Wired Headphones') 160
('Vareebadd Phone', 'Wired Headphones') 143
('Lightning Charging Cable', 'Wired Headphones') 92
